In [1]:
import numpy as np 
import matplotlib as mp
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
import math
import tensorflow.contrib.slim as slim
from sklearn.linear_model import LogisticRegression

In [2]:
def randomn_selection(X_train,label_train,k):
    idx = np.random.randint(X_train.shape[0], size=k)
    return X_train[idx],label_train[idx]

In [3]:
# Building the encoder
def encoder(x,weights,biases):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x,weights,biases):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

In [4]:
def trainEncoder(learning_rate,num_steps,batch_size,num_hidden_1,num_hidden_2,num_input,X_train,label_train):

    # tf Graph input (only pictures)
    X = tf.placeholder("float", [None, num_input])

    weights = {
        'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
        'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
        'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
        'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    }
    biases = {
        'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
        'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
        'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
        'decoder_b2': tf.Variable(tf.random_normal([num_input])),
    }
    
    # Construct model
    encoder_op = encoder(X,weights,biases)
    autoencoder = decoder(encoder_op,weights,biases)

    # Prediction
    y_pred = autoencoder
    # Targets (Labels) are the input data.
    y_true = X

    # Define loss and optimizer, minimize the squared error
    loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    
    # Start Training
    # Start a new TF session
    sess = tf.Session()

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, num_steps+1):
        # Prepare Data
        # Get the next batch of MNIST data (only images are needed, not labels)
        batch_x, _ = randomn_selection(X_train,label_train,batch_size)
        # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss], feed_dict={X: batch_x})
        # Display logs per step
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, l))
            
    return X,weights,biases,sess

In [5]:
text_file = open("/Users/panktibhalani/Documents/DM/HW2/spambase.txt", "r");
lines = text_file.readlines();
matrix = []
count = 0
labels = []
for line in lines:
    values = line.split(",")
    locallist = []
    for val in values:
        locallist.append(val)
    a = locallist[len(locallist) - 1].split()
    labels.append(a[0])
    del locallist[-1]
    locallist = list(map(float,locallist))
    matrix.append(locallist)
matrix=np.array(matrix)
print(np.array(matrix).shape)
hot_vector_labels = np.zeros((len(labels),2))

index = 0

X_trainSP, X_testSP, label_trainSP, label_testSP = train_test_split(np.array(matrix),np.array(labels) , test_size=0.20, random_state=42)


(4601, 57)


In [16]:
# Training Parameters
learning_rate = 0.01
num_steps = 10000
batch_size = 256
display_step = 1000
examples_to_show = 10

# Network Parameters
num_hidden_1 = 32 # 1st layer num features
num_hidden_2 = 16 # 2nd layer num features (the latent dim)
num_input = 57 # MNIST data input (img shape: 28*28)

X,weights,biases,sess = trainEncoder(learning_rate,num_steps,batch_size,num_hidden_1,num_hidden_2,num_input,X_trainSP,label_trainSP)


Step 1: Minibatch Loss: 8764.761719
Step 1000: Minibatch Loss: 11993.075195
Step 2000: Minibatch Loss: 7030.386719
Step 3000: Minibatch Loss: 7768.709473
Step 4000: Minibatch Loss: 5625.764160
Step 5000: Minibatch Loss: 4454.485352
Step 6000: Minibatch Loss: 12214.039062
Step 7000: Minibatch Loss: 5167.155762
Step 8000: Minibatch Loss: 13413.865234
Step 9000: Minibatch Loss: 12719.904297
Step 10000: Minibatch Loss: 6203.113281


In [17]:
encoedertemp =  encoder(X,weights,biases)
autoencoder = decoder(encoedertemp,weights,biases)

In [18]:
gtest = sess.run(encoedertemp, feed_dict={X: X_testSP})
gtrain = sess.run(encoedertemp, feed_dict={X: X_trainSP})
print(gtest.shape)
print(gtrain.shape)

(921, 16)
(3680, 16)


In [19]:
LR = LogisticRegression()
print(label_trainSP.shape)

LResult = LR.fit(gtrain,label_trainSP)
print(LResult.score(gtest,label_testSP))

(3680,)
0.9001085776330076


In [10]:
from sklearn.metrics.pairwise import euclidean_distances
distmatrix = euclidean_distances(gtest)
indexes = np.sort(np.argpartition(distmatrix,20,axis=1)[:,:20],axis=1)
indexes.shape

distmatrix = euclidean_distances(X_testSP)
indexestrain = np.sort(np.argpartition(distmatrix,20,axis=1)[:,:20],axis=1)
indexestrain.shape
idx = np.random.randint(indexestrain.shape[0], size=50)
selectedtest = indexestrain[idx]
selecttensor = indexes[idx]

majority = 0;

for i in range(50):
    print("SpamMail: " + str(i))
    resultold = ""
    
    for a in selectedtest[i]:
        tempdict[label_trainSP[a]] += 1
        resultold += " " + str(label_trainSP[a])
    tempmaj = 1
    if(tempdict['0'] > tempdict['1']):
        tempmaj = 0
    
    resultnew = ""
    
    for b in selecttensor[i]:
        resultnew += " " + str(label_testSP[b])
    print("OLD: " + resultold)
    print("Encoded: " + resultnew)
    print("\n")

SpamMail: 0
OLD:  1 1 1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0
Encoded:  1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0


SpamMail: 1
OLD:  1 1 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0
Encoded:  0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 1 1


SpamMail: 2
OLD:  0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1
Encoded:  1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0


SpamMail: 3
OLD:  0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 0 0 0 0
Encoded:  0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1


SpamMail: 4
OLD:  1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
Encoded:  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


SpamMail: 5
OLD:  0 1 0 1 0 0 1 1 1 0 1 0 0 0 0 1 0 1 0 0
Encoded:  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


SpamMail: 6
OLD:  1 1 0 1 1 0 1 1 1 0 0 0 1 0 0 0 1 0 0 0
Encoded:  0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1


SpamMail: 7
OLD:  0 1 0 0 1 1 1 1 0 1 1 0 1 0 1 0 0 0 1 0
Encoded:  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0


SpamMail: 8
OLD:  0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0
Encoded:  0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0


SpamMail: 